<a href="https://www.kaggle.com/code/klyushnik/classification-with-kears-tensorflow?scriptVersionId=168374849" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e3/sample_submission.csv
/kaggle/input/playground-series-s4e3/train.csv
/kaggle/input/playground-series-s4e3/test.csv
/kaggle/input/faulty-steel-plates/faults.csv


# Loading CSV

In [2]:
train = pd.read_csv('/kaggle/input/playground-series-s4e3/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s4e3/test.csv')
df_org = pd.read_csv('/kaggle/input/faulty-steel-plates/faults.csv')

print('Shape train is', train.shape, 'shape test is', test.shape)

Shape train is (19219, 35) shape test is (12814, 28)


In [3]:
train = pd.concat([train, df_org], ignore_index=True)
print('Shape train is', train.shape, 'shape test is', test.shape)

Shape train is (21160, 35) shape test is (12814, 28)


In [4]:
y_list = ["Pastry","Z_Scratch","K_Scatch","Stains","Dirtiness","Bumps","Other_Faults"]
print('Our proba_columns:', y_list)

Our proba_columns: ['Pastry', 'Z_Scratch', 'K_Scatch', 'Stains', 'Dirtiness', 'Bumps', 'Other_Faults']


In [5]:
train = train.replace((np.inf, -np.inf, np.nan), 0).reset_index(drop=True)

print('Shape train is', train.shape, 'shape test is', test.shape)

Shape train is (21160, 35) shape test is (12814, 28)


In [6]:
train = train.drop(columns = ['id'])
test = test.drop(columns = ['id'])

print('Shape train is', train.shape, 'shape test is', test.shape)

Shape train is (21160, 34) shape test is (12814, 27)


# Profiling Report

In [7]:
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)
import warnings
warnings.filterwarnings('ignore')
from IPython.display import Image
%matplotlib inline

In [8]:
# from ydata_profiling import ProfileReport

# ProfileReport(train, title="Profiling Report")

# Synthetic Data Vault

In [9]:
! pip install sdv

INFO: pip is looking at multiple versions of sdmetrics to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.6/125.6 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 92.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.7/170.7 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 67.0 MB/s eta 0:00:00
  Attempting uninstall: botocore
    Found existing installation: botocore 1.34.34
    Uninstalling botocore-1.34.34:
      Successfully uninstalled botocore-1.34.34
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source

In [10]:
from sdv.single_table import CTGANSynthesizer
import sdv
from sdv.single_table import GaussianCopulaSynthesizer
from sdv.metadata import SingleTableMetadata

In [11]:
metadata = SingleTableMetadata()
metadata.detect_from_dataframe(train)

In [12]:
metadata.validate_data(data=train)

In [13]:
synthesizer = GaussianCopulaSynthesizer(metadata)
synthesizer.fit(train)

synthetic_data = synthesizer.sample(num_rows=5000)

In [14]:
synthetic_data.shape

(5000, 34)

In [15]:
from sdv.evaluation.single_table import run_diagnostic

diagnostic = run_diagnostic(
    real_data=train,
    synthetic_data=synthetic_data,
    metadata=metadata
)

Generating report ...
(2/2) Evaluating Data Structure: : 100%|██████████| 1/1 [00:00<00:00, 343.29it/s]

Overall Score: 100.0%

Properties:
- Data Validity: 100.0%
- Data Structure: 100.0%


In [16]:
train = pd.concat([train, synthetic_data], ignore_index=True)

print('Shape train is', train.shape, 'shape test is', test.shape)

Shape train is (26160, 34) shape test is (12814, 27)


# Define X,y

In [17]:
X = train.drop(columns= y_list)
y = train[y_list]
X_total = test
work_list = X.columns
print(f'Shape X = {X.shape},shape y = {y.shape},shape X_total =, {X_total.shape}')

Shape X = (26160, 27),shape y = (26160, 7),shape X_total =, (12814, 27)


In [18]:
X_df = X.copy(deep = True)

X_df_test = X_total.copy(deep = True)

print(f'Shape X = {X_df.shape},shape y = {y.shape},shape X_total =, {X_df_test.shape}')

Shape X = (26160, 27),shape y = (26160, 7),shape X_total =, (12814, 27)


# Сonvert and add data

In [19]:
from sklearn.base import BaseEstimator, TransformerMixin
from category_encoders import OneHotEncoder, OrdinalEncoder, CountEncoder

In [20]:
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn import preprocessing

In [21]:
class AggFeatureExtractor(BaseEstimator, TransformerMixin):
    
    def __init__(self, group_col, agg_col, agg_func):
        self.group_col = group_col
        self.group_col_name = ''
        for col in group_col:
            self.group_col_name += col
        self.agg_col = agg_col
        self.agg_func = agg_func
        self.agg_df = None
        self.medians = None
        
    def fit(self, X, y=None):
        group_col = self.group_col
        agg_col = self.agg_col
        agg_func = self.agg_func
        
        self.agg_df = X.groupby(group_col)[agg_col].agg(agg_func)
        self.agg_df.columns = [f'{self.group_col_name}_{agg}_{_agg_col}' for _agg_col in agg_col for agg in agg_func]
        self.medians = X[agg_col].median()
        
        return self
    
    def transform(self, X):
        group_col = self.group_col
        agg_col = self.agg_col
        agg_func = self.agg_func
        agg_df = self.agg_df
        medians = self.medians
        
        X_merged = pd.merge(X, agg_df, left_on=group_col, right_index=True, how='left')
        X_merged.fillna(medians, inplace=True)
        X_agg = X_merged.loc[:, [f'{self.group_col_name}_{agg}_{_agg_col}' for _agg_col in agg_col for agg in agg_func]]
        
        return X_agg
    
    def fit_transform(self, X, y=None):
        self.fit(X, y)
        X_agg = self.transform(X)
        return X_agg

In [22]:
class Preprocessor:
    def __init__(self, agg_col, agg_func, group_cols):
        self.agg_col = agg_col
        self.agg_func = agg_func
        self.group_cols = group_cols
        
    def preprocess(self, X, X_total):    
        agg_train, agg_test = [], []
        for group_col in self.group_cols:
            agg_extractor = AggFeatureExtractor(group_col=group_col, agg_col=self.agg_col, agg_func=self.agg_func)
            agg_extractor.fit(pd.concat([X, X_total], axis=0))
            agg_train.append(agg_extractor.transform(X))
            agg_test.append(agg_extractor.transform(X_total))
        X = pd.concat([X] + agg_train, axis=1)
        X_total = pd.concat([X_total] + agg_test, axis=1)
        
        X = X.fillna(0)
        X_total = X_total.fillna(0)
        return X, X_total

In [23]:
agg_col = ['X_Minimum', 'X_Maximum', 'Y_Minimum', 'Y_Maximum', 'Pixels_Areas',
       'X_Perimeter', 'Y_Perimeter', 'Sum_of_Luminosity',
       'Minimum_of_Luminosity', 'Maximum_of_Luminosity', 'Length_of_Conveyer',
       'TypeOfSteel_A300', 'TypeOfSteel_A400', 'Steel_Plate_Thickness',
       'Edges_Index', 'Empty_Index', 'Square_Index', 'Outside_X_Index',
       'Edges_X_Index', 'Edges_Y_Index', 'Outside_Global_Index', 'LogOfAreas',
       'Log_X_Index', 'Log_Y_Index', 'Orientation_Index', 'Luminosity_Index',
       'SigmoidOfAreas']

agg_func = ['mean', 'std']

group_cols = [['X_Minimum'], ['X_Maximum'], ['Y_Minimum'], ['Y_Maximum'], ['Pixels_Areas'],
              ['Minimum_of_Luminosity'], ['Maximum_of_Luminosity'], ['Length_of_Conveyer'],
              ['TypeOfSteel_A300'], ['TypeOfSteel_A400'], ['Steel_Plate_Thickness'],
              ['Minimum_of_Luminosity', 'Maximum_of_Luminosity'], ['Length_of_Conveyer','TypeOfSteel_A300'],
              [ 'TypeOfSteel_A400', 'Steel_Plate_Thickness'],      
              ['Edges_Index', 'Empty_Index'],['Edges_X_Index', 'Edges_Y_Index'], 
              ['Outside_Global_Index', 'LogOfAreas'],[ 'Log_X_Index', 'Log_Y_Index'], 
              ['Orientation_Index', 'Luminosity_Index'],             
              ['Minimum_of_Luminosity', 'Maximum_of_Luminosity', 'Length_of_Conveyer'],
               ['TypeOfSteel_A300', 'TypeOfSteel_A400', 'Steel_Plate_Thickness'],
               ['Edges_Index', 'Empty_Index', 'Square_Index', 'Outside_X_Index'],
               ['Edges_X_Index', 'Edges_Y_Index', 'Outside_Global_Index', 'LogOfAreas'],
               ['Log_X_Index', 'Log_Y_Index', 'Orientation_Index', 'Luminosity_Index']]

pp = Preprocessor(agg_col, agg_func, group_cols)
X, X_total = pp.preprocess(X, X_total)

print(f"X shape :{X.shape} , y shape :{y.shape}, X_test shape :{X_total.shape}")

X shape :(26160, 1323) , y shape :(26160, 7), X_test shape :(12814, 1323)


In [24]:
transform = PowerTransformer(method='yeo-johnson')
transform2 = QuantileTransformer(n_quantiles=10, random_state=0)

for i in work_list[1:]:
    X[i+'+log'] = (X[i]+1).transform(np.log)
    X_total[i+'+log'] =(X_total[i]+1).transform(np.log)
    
    X[i+'+log1'] = (X[i]+1).transform(np.log1p)
    X_total[i+'+log1'] =(X_total[i]+1).transform(np.log1p)
    
    X[i+'+y_j'] = transform.fit_transform(X[[i]])
    X_total[i+'+y_j'] = transform.fit_transform(X_total[[i]])
    
    X[i+'+q_t'] = transform2.fit_transform(X[[i]])
    X_total[i+'+q_t'] = transform2.fit_transform(X_total[[i]])
       
    X[i+'+sqrt'] = (X[i]+1).transform(np.sqrt)
    X_total[i+'+sqrt'] =(X_total[i]+1).transform(np.sqrt)
    
print(f"X shape :{X.shape} , y shape :{y.shape}, X_test shape :{X_total.shape}")

X shape :(26160, 1453) , y shape :(26160, 7), X_test shape :(12814, 1453)


# Polynomial Features

In [25]:
def PolynomialFeatures_labeled(input_df,power):
   
    poly = preprocessing.PolynomialFeatures(power)
    output_nparray = poly.fit_transform(input_df)
    powers_nparray = poly.powers_

    input_feature_names = list(input_df.columns)
    target_feature_names = ["Constant Term"]
    for feature_distillation in powers_nparray[1:]:
        intermediary_label = ""
        final_label = ""
        for i in range(len(input_feature_names)):
            if feature_distillation[i] == 0:
                continue
            else:
                variable = input_feature_names[i]
                power = feature_distillation[i]
                intermediary_label = "%s+%d" % (variable,power)
                if final_label == "":         #If the final label isn't yet specified
                    final_label = intermediary_label
                else:
                    final_label = final_label + "x" + intermediary_label
        target_feature_names.append(final_label)
    output_df = pd.DataFrame(output_nparray, columns = target_feature_names)
    return output_df

output_df = PolynomialFeatures_labeled(X[work_list[1:]],2)
output_df_t = PolynomialFeatures_labeled(X_total[work_list[1:]],2)

X =  pd.concat([X.reset_index(drop=True), output_df], axis = 1)
X_total = pd.concat([X_total.reset_index(drop=True), output_df_t], axis = 1)


print(f"X shape :{X.shape} , y shape :{y.shape}, X_test shape :{X_total.shape}")

X shape :(26160, 1831) , y shape :(26160, 7), X_test shape :(12814, 1831)


In [26]:
X = X.replace((np.inf, -np.inf, np.nan), 0).reset_index(drop=True)
X_total = X_total.replace((np.inf, -np.inf, np.nan), 0).reset_index(drop=True)

print(f"X shape :{X.shape} , y shape :{y.shape}, X_test shape :{X_total.shape}")

X shape :(26160, 1831) , y shape :(26160, 7), X_test shape :(12814, 1831)


# Threshold

In [27]:
from sklearn.feature_selection import VarianceThreshold
def variance_threshold(df,th):
    var_thres=VarianceThreshold(threshold=th)
    var_thres.fit(df)
    new_cols = var_thres.get_support()
    return df.iloc[:,new_cols]

In [28]:
X = variance_threshold(X,0)
list_name = (X.columns)
X_total = X_total[list_name]

print(f"X shape :{X.shape} , y shape :{y.shape}, X_test shape :{X_total.shape}")

X shape :(26160, 1784) , y shape :(26160, 7), X_test shape :(12814, 1784)


# PCA

In [29]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
principalComponents = pca.fit_transform(X)
principalComponents_test = pca.fit_transform(X_total)

principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2'])
principalDftest = pd.DataFrame(data = principalComponents_test
             , columns = ['principal component 1', 'principal component 2'])

X = pd.concat([X_df, principalDf], axis = 1)
X_total = pd.concat([X_df_test, principalDftest], axis = 1)

print(f"X shape :{X.shape} , y shape :{y.shape}, X_test shape :{X_total.shape}")

X shape :(26160, 29) , y shape :(26160, 7), X_test shape :(12814, 29)


# StandardScaler

In [30]:
from sklearn.preprocessing import StandardScaler

std = StandardScaler()

X = std.fit_transform(X)
X_total = std.fit_transform(X_total)
y = std.fit_transform(y)

print(f"X shape :{X.shape} , y shape :{y.shape}, X_test shape :{X_total.shape}")

X shape :(26160, 29) , y shape :(26160, 7), X_test shape :(12814, 29)


# Build Model

In [31]:
import tensorflow as tf
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver() # TPU detection
except ValueError:
    tpu = None
    gpus = tf.config.experimental.list_logical_devices("GPU")
    
if tpu:
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu,) 
    print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
elif len(gpus) > 1:
    strategy = tf.distribute.MirroredStrategy([gpu.name for gpu in gpus])
    print('Running on multiple GPUs ', [gpu.name for gpu in gpus])
elif len(gpus) == 1:
    strategy = tf.distribute.get_strategy() 
    print('Running on single GPU ', gpus[0].name)
else:
    strategy = tf.distribute.get_strategy() 
    print('Running on CPU')
print("Number of accelerators: ", strategy.num_replicas_in_sync)
from tensorflow.keras import Sequential
from tensorflow.keras.initializers import Constant
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers

2024-03-23 05:07:28.090012: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-23 05:07:28.090114: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-23 05:07:28.217135: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Running on single GPU  /device:GPU:0
Number of accelerators:  1


In [32]:
from tensorflow import keras
from tensorflow.keras import layers

In [33]:
input_shape = [X.shape[1]]

In [34]:
model = keras.Sequential([  
    layers.BatchNormalization(input_shape=input_shape),
    
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),

    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
   
    layers.Dropout(0.2),
    layers.Dense(7, activation='softmax'),
])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ batch_normalization             │ (None, 29)             │           116 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 29)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │         7,680 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 7)              │         1,799 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 144,251 (563.48 KB)

 Trainable params: 142,657 (557.25 KB)

 Non-trainable params: 1,594 (6.23 KB)

In [35]:
from sklearn.model_selection import StratifiedKFold, KFold, StratifiedGroupKFold, RepeatedStratifiedKFold,RepeatedKFold, cross_validate

In [36]:
folds = 5

for kfold, (train, test) in enumerate(KFold(n_splits=folds, 
                                shuffle=True).split(X, y)):
    print(f"Training and evaluating on fold {kfold+1} out of {folds}...")
    tf.keras.backend.clear_session()    
    model.compile(loss  = 'categorical_crossentropy',
                  metrics  = ['accuracy'],
                  optimizer = keras.optimizers.Adam(learning_rate=1e-4))
    print('Train Set')
    print(X[train].shape)
    print(y[train].shape)

    print('Test Set')
    print(X[test].shape)
    print(y[test].shape)
    
    early_stopping = keras.callbacks.EarlyStopping(patience=5,
                                                   min_delta=0.001,
                                                   restore_best_weights=True,)
    reduce_lr = keras.callbacks.ReduceLROnPlateau(factor = 0.1, patience = 2, mode = 'min', verbose = 1,)
    
    model.fit(X[train], y[train],
              batch_size=150,
              epochs=200, 
              validation_data=(X[test], y[test]),
              callbacks=[reduce_lr,early_stopping],)
    scores = model.evaluate(X[test], y[test], verbose=0)
    
    print(scores)

Training and evaluating on fold 1 out of 5...
Train Set
(20928, 29)
(20928, 7)
Test Set
(5232, 29)
(5232, 7)
Epoch 1/200
 63/140 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1718 - loss: -0.8932

I0000 00:00:1711170468.273371      89 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


140/140 ━━━━━━━━━━━━━━━━━━━━ 19s 60ms/step - accuracy: 0.2075 - loss: -1.6243 - val_accuracy: 0.4130 - val_loss: -3.3019 - learning_rate: 1.0000e-04
Epoch 2/200
140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3445 - loss: -6.9230 - val_accuracy: 0.4195 - val_loss: -10.5550 - learning_rate: 1.0000e-04
Epoch 3/200
140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3741 - loss: -12.2297 - val_accuracy: 0.4213 - val_loss: -18.3683 - learning_rate: 1.0000e-04
Epoch 4/200
140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3862 - loss: -17.1655 - val_accuracy: 0.4193 - val_loss: -24.7308 - learning_rate: 1.0000e-04
Epoch 5/200
140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3964 - loss: -22.5130 - val_accuracy: 0.4170 - val_loss: -29.8825 - learning_rate: 1.0000e-04
Epoch 6/200
140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4014 - loss: -27.2646 - val_accuracy: 0.4188 - val_loss: -34.9552 - learning_rate: 1.0000e-04
Epoch 7/200
140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms

# Submit

In [37]:
predict_tensor=model.predict(X_total)
print(f'shape of {predict_tensor.shape}')

401/401 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
shape of (12814, 7)


In [38]:
sample = pd.read_csv('/kaggle/input/playground-series-s4e3/sample_submission.csv')
my_sample =pd.DataFrame(predict_tensor, columns=[y_list])
my_sample['id'] = sample['id']
my_sample

,Pastry,Z_Scratch,K_Scatch,Stains,Dirtiness,Bumps,Other_Faults,id
0,0.107360,0.000000e+00,0.0,0.0,1.054708e-19,0.077887,0.814753,19219
1,0.324627,0.000000e+00,0.0,0.0,2.093403e-15,0.012788,0.662584,19220
2,0.069867,0.000000e+00,0.0,0.0,7.062850e-05,0.000150,0.929913,19221
3,0.050144,0.000000e+00,0.0,0.0,1.991395e-13,0.007030,0.942827,19222
4,0.051153,0.000000e+00,0.0,0.0,3.107575e-08,0.000623,0.948224,19223
...,...,...,...,...,...,...,...,...
12809,0.063468,2.431640e-39,0.0,0.0,8.408263e-12,0.003852,0.932680,32028
12810,0.437047,0.000000e+00,0.0,0.0,3.140803e-18,0.061153,0.501800,32029
12811,0.000000,0.000000e+00,1.0,0.0,0.000000e+00,0.000000,0.000000,32030
12812,0.431215,0.000000e+00,0.0,0.0,1.105823e-16,0.018572,0.550213,32031


In [39]:
my_sample.to_csv('submission.csv', index=False)